In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\anime-faces\data"):
            for file in f:
                if '.png' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = np.array(img)
            x = x[...,:3]
            if(x.shape == (64,64,3)):
                data.append(x)

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),64,64,3)
        return x_train

In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 50

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(64 * 4 * 4, activation="elu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 64)))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu"))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same", activation="elu"))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same", activation="elu"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save(r"C:\Users\Vee\Desktop\python\GAN\models\generator" + str(epoch) + ".h5")

In [4]:
dcgan = DCGAN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)       

In [ ]:
dcgan.train(epochs=50001, batch_size=150, save_interval=100)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0

0 [D loss: 1.141820, acc.: 30.67%] [G loss: 0.861057]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.629642, acc.: 63.33%] [G loss: 1.314801]
2 [D loss: 0.521191, acc.: 72.33%] [G loss: 1.585969]
3 [D loss: 0.437068, acc.: 80.33%] [G loss: 1.966406]
4 [D loss: 0.347702, acc.: 86.33%] [G loss: 2.033294]
5 [D loss: 0.379780, acc.: 84.67%] [G loss: 1.924361]
6 [D loss: 0.309203, acc.: 87.00%] [G loss: 2.300166]
7 [D loss: 0.254275, acc.: 92.67%] [G loss: 2.610265]
8 [D loss: 0.309840, acc.: 90.67%] [G loss: 2.536473]
9 [D loss: 0.284095, acc.: 91.33%] [G loss: 2.658755]
10 [D loss: 0.239553, acc.: 93.00%] [G loss: 2.777849]
11 [D loss: 0.273967, acc.: 90.33%] [G loss: 2.745204]
12 [D loss: 0.275030, acc.: 90.33%] [G loss: 2.718717]
13 [D loss: 0.275173, acc.: 90.00%] [G loss: 2.870165]
14 [D loss: 0.291254, acc.: 88.33%] [G loss: 2.959060]
15 [D loss: 0.258585, acc.: 91.00%] [G loss: 2.839783]
16 [D loss: 0.372760, acc.: 83.67%] [G loss: 2.686062]
17 [D loss: 0.390908, acc.: 81.67%] [G loss: 3.134585]
18 [D loss: 0.343490, acc.: 87.33%] [G loss: 2.894530]
19 [D loss: 0.38823

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

13301 [D loss: 0.312144, acc.: 87.33%] [G loss: 3.645623]
13302 [D loss: 0.327359, acc.: 85.33%] [G loss: 3.829506]
13303 [D loss: 0.294869, acc.: 87.67%] [G loss: 3.842474]
13304 [D loss: 0.419524, acc.: 78.67%] [G loss: 4.143905]
13305 [D loss: 0.439742, acc.: 79.67%] [G loss: 2.917686]
13306 [D loss: 0.325122, acc.: 86.00%] [G loss: 3.274592]
13307 [D loss: 0.468495, acc.: 78.00%] [G loss: 3.505182]
13308 [D loss: 0.441913, acc.: 78.67%] [G loss: 4.000182]
13309 [D loss: 0.315881, acc.: 88.67%] [G loss: 3.769307]
13310 [D loss: 0.321288, acc.: 87.33%] [G loss: 3.353231]
13311 [D loss: 0.289704, acc.: 86.00%] [G loss: 4.687643]
13312 [D loss: 0.193548, acc.: 93.00%] [G loss: 3.849359]
13313 [D loss: 0.283616, acc.: 88.33%] [G loss: 3.378491]
13314 [D loss: 0.315933, acc.: 85.67%] [G loss: 3.768164]
13315 [D loss: 0.300113, acc.: 89.67%] [G loss: 3.371184]
13316 [D loss: 0.574149, acc.: 72.67%] [G loss: 3.584869]
13317 [D loss: 0.526075, acc.: 76.00%] [G loss: 2.986573]
13318 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


13399 [D loss: 0.266823, acc.: 91.00%] [G loss: 3.607037]
13400 [D loss: 0.276999, acc.: 88.33%] [G loss: 3.701038]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


13401 [D loss: 0.255819, acc.: 91.00%] [G loss: 3.252750]
13402 [D loss: 0.352809, acc.: 84.67%] [G loss: 3.171856]
13403 [D loss: 0.552507, acc.: 71.67%] [G loss: 3.357606]
13404 [D loss: 0.345087, acc.: 85.33%] [G loss: 4.091669]
13405 [D loss: 0.275829, acc.: 90.00%] [G loss: 3.932912]
13406 [D loss: 0.398807, acc.: 82.67%] [G loss: 3.249143]
13407 [D loss: 0.554291, acc.: 73.33%] [G loss: 2.882553]
13408 [D loss: 0.568276, acc.: 71.67%] [G loss: 3.309879]
13409 [D loss: 0.656175, acc.: 65.00%] [G loss: 2.953835]
13410 [D loss: 0.400575, acc.: 78.67%] [G loss: 3.351722]
13411 [D loss: 0.402313, acc.: 79.00%] [G loss: 4.496964]
13412 [D loss: 0.468974, acc.: 76.67%] [G loss: 4.112407]
13413 [D loss: 0.204454, acc.: 91.33%] [G loss: 4.808836]
13414 [D loss: 0.430732, acc.: 80.33%] [G loss: 3.280191]
13415 [D loss: 0.217304, acc.: 93.67%] [G loss: 3.466954]
13416 [D loss: 0.169847, acc.: 95.33%] [G loss: 3.445955]
13417 [D loss: 0.249845, acc.: 91.00%] [G loss: 3.349798]
13418 [D loss: